In [1]:
import csv
import itertools

DataFile = open('Depression.csv', 'r')
minsup = 0.03
f2 = "Rules.txt"
f1 = "FItems.txt"
minconf = 0.39


def L1():
    DataCaptured = csv.reader(DataFile, delimiter=',')
    data = list(DataCaptured)
    for e in data:
        e = sorted(e)
    count = {}
    for items in data:
        for item in items:
            if item not in count:
                count[(item)] = 1
            else:
                count[(item)] = count[(item)] + 1
    #print("C1 Items", count)
    print("C1 Length : ", len(count))
    print()

    #Thresholding
    count2 = {k: v for k, v in count.items() if v >= minsup*9835}
    #print("L1 Items : ", count2)
    print("L1 Length : ", len(count2))
    print()

    return count2, data


def generateCk(Lk_1, flag, data):
    Ck = []

    if flag == 1:
        flag = 0
        for item1 in Lk_1:
            for item2 in Lk_1:
                if item2 > item1:
                    Ck.append((item1, item2))
        print("C2: ", Ck[1:3])
        print("length : ", len(Ck))
        print()

    else:
        for item in Lk_1:
            k = len(item)
        for item1 in Lk_1:
            for item2 in Lk_1:
                if (item1[:-1] == item2[:-1]) and (item1[-1] != item2[-1]):
                    if item1[-1] > item2[-1]:
                        Ck.append(item2 + (item1[-1],))
                    else:
                        Ck.append(item1 + (item2[-1],))
        print("C" + str(k+1) + ": ", Ck[1:3])
        print("Length : ", len(Ck))
        print()
    L = generateLk(set(Ck), data)
    return L, flag


def generateLk(Ck, data):
    count = {}
    for itemset in Ck:
        #print(itemset)
        for transaction in data:
            if all(e in transaction for e in itemset):
                if itemset not in count:
                    count[itemset] = 1
                else:
                    count[itemset] = count[itemset] + 1

    print("Ct Length : ", len(count))
    print()

    count2 = {k: v for k, v in count.items() if v >= minsup*9835}
    print("L Length : ", len(count2))
    print()
    return count2


def rulegenerator(fitems):
    counter = 0
    for itemset in fitems.keys():
        if isinstance(itemset, str):
            continue
        length = len(itemset)

        union_support = fitems[tuple(itemset)]
        for i in range(1, length):

            lefts = map(list, itertools.combinations(itemset, i))
            for left in lefts:
                if len(left) == 1:
                    if ''.join(left) in fitems:
                        leftcount = fitems[''.join(left)]
                        conf = union_support / leftcount
                else:
                    if tuple(left) in fitems:
                        leftcount = fitems[tuple(left)]
                        conf = union_support / leftcount
                if conf >= minconf:
                    fo = open(f2, "a+")
                    right = list(itemset[:])
                    for e in left:
                        right.remove(e)
                    fo.write(str(left) + ' (' + str(leftcount) + ')' + ' -> ' + str(right) + ' (' + str(fitems[''.join(right)]) + ')' + ' [' + str(conf) + ']' + '\n')
                    print(str(left) + ' -> ' + str(right) + ' (' + str(conf) + ')')
                    counter = counter + 1
                    #Greater than 1???
                    fo.close()
    print(counter, "rules generated")


def apriori():
    '''
    The runner function
    '''
    L, data = L1()
    flag = 1
    FreqItems = dict(L)
    while(len(L) != 0):
        fo = open(f1, "a+")
        for k, v in L.items():
            fo.write(str(k) + ' >>> ' + str(v) + '\n\n')
        fo.close()

        L, flag = generateCk(L, flag, data)
        FreqItems.update(L)
    rulegenerator(FreqItems)

apriori()

C1 Length :  988

L1 Length :  12

C2:  [('Good', 'Hall-Mess'), ('Good', 'Single')]
length :  66

Ct Length :  64

L Length :  22

C3:  [('5', 'No', 'Yes'), ('5', 'Male', 'No')]
Length :  32

Ct Length :  16

L Length :  15

C4:  [('Bad', 'No', 'No one', 'Yes'), ('Male', 'No', 'Single', 'Yes')]
Length :  6

Ct Length :  3

L Length :  3

C5:  []
Length :  0

Ct Length :  0

L Length :  0

['No one'] -> ['Yes'] (1.0)
['Hall-Mess'] -> ['No'] (1.0)
['5'] -> ['No'] (1.0)
['Female'] -> ['No'] (1.0)
['Male'] -> ['Single'] (0.5893186003683242)
['Single'] -> ['Male'] (0.8839779005524862)
['Hall-Mess'] -> ['Yes'] (1.0)
['Male'] -> ['No'] (1.0)
['Single'] -> ['Yes'] (1.0)
['5'] -> ['Yes'] (1.0)
['Bad'] -> ['No'] (0.6491228070175439)
['Good'] -> ['No'] (0.7139830508474576)
['Bad'] -> ['No one'] (0.6368421052631579)
['No one'] -> ['Bad'] (0.9945205479452055)
['Female'] -> ['Yes'] (1.0)
['3'] -> ['No'] (1.0)
['Male'] -> ['Yes'] (1.0)
['Single'] -> ['No'] (1.0)
['5'] -> ['Male'] (0.6842105263157895)

KeyError: 'MaleNo'